https://www.youtube.com/watch?v=l99baVWtf30&list=PLZ1QII7yudbecO6a-zAI6cuGP1LLnmW8e&index=15

In [117]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [118]:
pair = "EUR_USD"
granularity = "H1"
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [119]:

df = pd.read_csv(utils.get_hist_data_filename(pair, granularity), index_col=False)

We want most of the columns in our dataframe to be numeric so that we can do maths.  non_cols is the only columns where we don't want to do this.

In [120]:
non_cols = ["time", "volume", "ticker"]
mod_cols = [x for x in df.columns if x not in non_cols]

In [121]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [122]:
df_plot = df.iloc[-500:].copy()

First use(s) of plotly library.  Note: "go" acronym references 'graph_objects'.  Not to be confused with Golang - which I've also been playing with recently.  It's all fun!!

In [123]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low = df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
    ))
fig.update_layout(width=1000, height=400, paper_bgcolor = "#1e1e1e", plot_bgcolor = "#1e1e1e",
                margin=dict(l=10, b=10, t=10, r=10), 
                font=dict(size=10, color="#e1e1e1"))
fig.update_xaxes(gridcolor="#1f292f",
                 showgrid=True,
                 fixedrange=True,
                 rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor="#1f292f",
                 showgrid=True)
fig.show()

OK, so far so good.  Time to start building some indicators, starting with Moving Average (MA)

In [124]:
df_ma = df[["time", "ticker", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()
df_ma.shape

ma_list = [16, 64]
for ma in ma_list:
    MA_Period = f"MA_{ma}"
    df_ma[MA_Period] = df_ma.mid_c.rolling(window=ma).mean()

df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [125]:
df_ma.head()

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2022-11-25T11:00:00.000000000Z,EUR_USD,1.04148,1.04148,1.03889,1.03901,1.041048,1.038329
1,2022-11-25T12:00:00.000000000Z,EUR_USD,1.03900,1.03976,1.03759,1.03788,1.040830,1.038451
2,2022-11-25T13:00:00.000000000Z,EUR_USD,1.03788,1.03835,1.03612,1.03728,1.040586,1.038561
3,2022-11-25T14:00:00.000000000Z,EUR_USD,1.03729,1.03888,1.03548,1.03816,1.040417,1.038682
4,2022-11-25T15:00:00.000000000Z,EUR_USD,1.03816,1.04072,1.03766,1.04068,1.040412,1.038832


In [126]:
df_plot = df_ma.iloc[-4000:].copy()

In [127]:
df_plot = df_ma.iloc[-500:].copy()

for ma in ma_list:
    ma_period = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[ma_period], 
        line=dict(width=2),
        line_shape="spline",
        name=ma_period))

fig.show()


In [128]:
df_ma = df_plot

In [129]:
df_ma["Diff"] = df_ma.MA_16 - df_ma.MA_64

In [130]:
df_ma.head()

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,Diff
3436,2023-06-15T15:00:00.000000000Z,EUR_USD,1.09103,1.09358,1.09098,1.09328,1.084321,1.081424,0.002897
3437,2023-06-15T16:00:00.000000000Z,EUR_USD,1.09327,1.09452,1.09322,1.09438,1.084981,1.081710,0.003270
3438,2023-06-15T17:00:00.000000000Z,EUR_USD,1.09436,1.09502,1.09396,1.09430,1.085806,1.081984,0.003822
3439,2023-06-15T18:00:00.000000000Z,EUR_USD,1.09432,1.09512,1.09416,1.09504,1.086662,1.082259,0.004403
3440,2023-06-15T19:00:00.000000000Z,EUR_USD,1.09504,1.09527,1.09452,1.09478,1.087502,1.082523,0.004979


In [131]:
df_ma["Diff_Prev"] = df_ma.Diff.shift(1)

In [132]:
df_ma.head()

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,Diff,Diff_Prev
3436,2023-06-15T15:00:00.000000000Z,EUR_USD,1.09103,1.09358,1.09098,1.09328,1.084321,1.081424,0.002897,NaN
3437,2023-06-15T16:00:00.000000000Z,EUR_USD,1.09327,1.09452,1.09322,1.09438,1.084981,1.081710,0.003270,0.002897
3438,2023-06-15T17:00:00.000000000Z,EUR_USD,1.09436,1.09502,1.09396,1.09430,1.085806,1.081984,0.003822,0.003270
3439,2023-06-15T18:00:00.000000000Z,EUR_USD,1.09432,1.09512,1.09416,1.09504,1.086662,1.082259,0.004403,0.003822
3440,2023-06-15T19:00:00.000000000Z,EUR_USD,1.09504,1.09527,1.09452,1.09478,1.087502,1.082523,0.004979,0.004403


In [133]:
def is_trade(row):
    if row.Diff >= 0 and row.Diff_Prev < 0:
        return 1
    if row.Diff <= 0 and row.Diff_Prev > 0:
        return -1
    return 0

In [134]:
df_ma["IS_TRADE"] = df_ma.apply(is_trade, axis=1)

In [135]:
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [136]:
df_trades.head()

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,Diff,Diff_Prev,IS_TRADE
3490,2023-06-19T21:00:00.000000000Z,EUR_USD,1.09226,1.09248,1.09206,1.09229,1.092157,1.092384,-0.000227,0.000044,-1
3533,2023-06-21T16:00:00.000000000Z,EUR_USD,1.09506,1.09738,1.09497,1.09706,1.092537,1.092304,0.000234,-0.000055,1
3570,2023-06-23T05:00:00.000000000Z,EUR_USD,1.09331,1.09341,1.09273,1.09326,1.095027,1.095062,-0.000035,0.000294,-1
3619,2023-06-27T06:00:00.000000000Z,EUR_USD,1.09176,1.09336,1.09162,1.09311,1.091456,1.091286,0.000170,-0.000018,1
3657,2023-06-28T20:00:00.000000000Z,EUR_USD,1.09161,1.09170,1.09113,1.09128,1.093011,1.093067,-0.000056,0.000184,-1


In [137]:
df_trades.shape

(8, 11)

In [138]:
df_trades["Delta"] = df_trades.mid_c.diff()
df_trades.head()

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,Diff,Diff_Prev,IS_TRADE,Delta
3490,2023-06-19T21:00:00.000000000Z,EUR_USD,1.09226,1.09248,1.09206,1.09229,1.092157,1.092384,-0.000227,0.000044,-1,NaN
3533,2023-06-21T16:00:00.000000000Z,EUR_USD,1.09506,1.09738,1.09497,1.09706,1.092537,1.092304,0.000234,-0.000055,1,0.00477
3570,2023-06-23T05:00:00.000000000Z,EUR_USD,1.09331,1.09341,1.09273,1.09326,1.095027,1.095062,-0.000035,0.000294,-1,-0.00380
3619,2023-06-27T06:00:00.000000000Z,EUR_USD,1.09176,1.09336,1.09162,1.09311,1.091456,1.091286,0.000170,-0.000018,1,-0.00015
3657,2023-06-28T20:00:00.000000000Z,EUR_USD,1.09161,1.09170,1.09113,1.09128,1.093011,1.093067,-0.000056,0.000184,-1,-0.00183


In [139]:
df_trades["PIP_Delta"] = (df_trades.mid_c.diff() /  i_pair.ins_pipLocation).shift(-1)
df_trades.head()

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,Diff,Diff_Prev,IS_TRADE,Delta,PIP_Delta
3490,2023-06-19T21:00:00.000000000Z,EUR_USD,1.09226,1.09248,1.09206,1.09229,1.092157,1.092384,-0.000227,0.000044,-1,NaN,47.7
3533,2023-06-21T16:00:00.000000000Z,EUR_USD,1.09506,1.09738,1.09497,1.09706,1.092537,1.092304,0.000234,-0.000055,1,0.00477,-38.0
3570,2023-06-23T05:00:00.000000000Z,EUR_USD,1.09331,1.09341,1.09273,1.09326,1.095027,1.095062,-0.000035,0.000294,-1,-0.00380,-1.5
3619,2023-06-27T06:00:00.000000000Z,EUR_USD,1.09176,1.09336,1.09162,1.09311,1.091456,1.091286,0.000170,-0.000018,1,-0.00015,-18.3
3657,2023-06-28T20:00:00.000000000Z,EUR_USD,1.09161,1.09170,1.09113,1.09128,1.093011,1.093067,-0.000056,0.000184,-1,-0.00183,-11.2


In [140]:
df_trades["Gain"] = df_trades.PIP_Delta * df_trades.IS_TRADE

In [141]:
df_trades.Gain.sum()

-117.10000000000109